# Import Libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

2025-09-26 05:03:29.068135: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)
!python -V

2.16.2
Python 3.11.0


# Download the Shakespeare dataset

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
            'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',
            cache_dir = '.'
        )

# Read the data

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [7]:
vocab[0:10]

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']

# Process the text

In [8]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [10]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [11]:
ids_from_chars.get_vocabulary()[0:10]

['[UNK]', '\n', ' ', '!', '$', '&', "'", ',', '-', '.']

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [13]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Create trainig examples and targets

In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
for ids in ids_dataset.take(5):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t


2025-09-26 05:03:36.801390: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [19]:
seq_length = 100

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(2):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)
tf.Tensor(
[b'a' b'r' b'e' b' ' b'a' b'l' b'l' b' ' b'r' b'e' b's' b'o' b'l' b'v'
 b'e' b'd' b' ' b'r' b'a' b't' b'h' b'e' b'r' b' ' b't' b'o' b' ' b'd'
 b'i' b'e' b' ' b't' b'h' b'a' b'n' b' ' b't' b'o' b' ' b'f' b'a' b'm'
 b'i' b's' b'h' b'?' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'R' b'e' b's'
 b'o' b'l' b'v' b'e' b'd' b'.' b' ' b'r' b'e' b's' b'o' b'l' b'v' b'e'
 b'd' b'.' b'\n' b'\n' b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'

2025-09-26 05:03:36.837908: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


2025-09-26 05:03:36.860795: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


2025-09-26 05:03:37.013127: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Create Training batches

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# Build the Model

In [27]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                        return_sequences=True,
                                        return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        # print("x.shape: ", x.shape)
        if states is None:
            states = self.gru.get_initial_state(tf.shape(x)[0])
            # print("states.shape: ", states.shape)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [29]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    # print("input_example_batch: ", input_example_batch[0:1])
    # print("target_example_batch: ", target_example_batch[0:1])
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


2025-09-26 05:03:40.906106: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [31]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 256)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ((64, 100, 1024), (64, │     3,938,304 │
│                                 │ 1024))                 │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 66)          │        67,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 66), dtype=float32, numpy=
array([[-0.00012413, -0.0022905 , -0.00244431, ...,  0.00544687,
         0.00761456,  0.00187808],
       [-0.0012801 ,  0.00982413, -0.00591877, ...,  0.00754811,
         0.00640314,  0.00526765],
       [ 0.00485846,  0.00342256, -0.00914048, ..., -0.00179751,
        -0.00410878, -0.00780774],
       ...,
       [ 0.01304836,  0.00219803, -0.01248215, ...,  0.00355694,
         0.01455144,  0.00172605],
       [ 0.00825678,  0.00715505, -0.00620926, ...,  0.00715853,
         0.01623205, -0.00069848],
       [ 0.00381869,  0.00278276, -0.00525054, ...,  0.0103658 ,
         0.01606963,  0.00148863]], dtype=float32)>

In [33]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([10,  1, 50, 43, 28, 49,  7, 12, 39, 16, 41, 30, 31, 31, 47, 36, 14,
       21,  4, 57, 53, 51, 21, 16, 48, 57, 59,  8, 56, 15, 42, 59, 28, 38,
       53, 14, 51, 33,  0,  4, 46,  6, 40, 65, 49, 33, 38, 12, 36, 15, 44,
       19,  4, 57, 39, 23, 37, 38, 41, 24, 42,  1, 62, 41,  1,  1, 50, 55,
        0, 46, 50, 59, 16, 28,  5,  9, 50, 52, 18,  7, 10, 21, 10, 46, 47,
       51, 16,  5, 44, 65,  8, 60, 36, 31, 57, 50,  9,  4, 41,  6])

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"th:\nThe George, profaned, hath lost his holy honour;\nThe garter, blemish'd, pawn'd his knightly virt"

Next Char Predictions:
 b"3\nkdOj,;ZCbQRRhWAH$rnlHCirt-qBctOYnAlT[UNK]$g'azjTY;WBeF$rZJXYbKc\nwb\n\nkp[UNK]gktCO&.kmE,3H3ghlC&ez-uWRrk.$b'"


# Train the Model

In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.18909, shape=(), dtype=float32)


In [37]:
tf.exp(example_batch_mean_loss).numpy()

65.96272

In [38]:
model.compile(optimizer='adam', loss=loss)

In [39]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 304s 2s/step - loss: 2.4984
Epoch 2/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - loss: 1.8284
Epoch 3/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 351s 2s/step - loss: 1.5902
Epoch 4/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 347s 2s/step - loss: 1.4650
Epoch 5/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - loss: 1.3878
Epoch 6/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - loss: 1.3312
Epoch 7/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 350s 2s/step - loss: 1.2848
Epoch 8/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step - loss: 1.2437
Epoch 9/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 360s 2s/step - loss: 1.2049
Epoch 10/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - loss: 1.1665
Epoch 11/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - loss: 1.1280
Epoch 12/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - loss: 1.0868
Epoch 13/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 304s 2s/step - loss: 1.0456
Epoch 14/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 307s 2s/step - loss: 1.0000
Epoch 15/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 31

# Generate Text

In [40]:
skip_ids = ids_from_chars(['[UNK]'])
skip_ids

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [41]:
skip_ids = ids_from_chars(['[UNK]'])[:, None]
skip_ids

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[0]])>

In [42]:
len(ids_from_chars.get_vocabulary())

66

In [43]:
len(skip_ids)

1

In [44]:
values=[-float('inf')]*len(skip_ids)
values

[-inf]

In [45]:
sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
tf.sparse.to_dense(sparse_mask)
    

<tf.Tensor: shape=(66,), dtype=float32, numpy=
array([-inf,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
      dtype=float32)>

In [46]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    # print("input_chars: ", input_chars)
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    # print("input_ids: ", input_ids)

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask
    # print("predicted_logits: ", predicted_logits)

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)
    # print("predicted_chars: ", predicted_chars)

    # Return the characters and model state.
    return predicted_chars, states

In [47]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [48]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
result
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time (sec):', end - start)

ROMEO:
Let me but rather; we'll remain alike.

First Watchman:
Why in this one heigh, now you can never be obey:
Could be a bawd; and therefore give me leave
To come was fat gone rose from quench and thrifts;
And so mightst me in honour both: we shall not spoke it,
Mistake to stay.

PETRUCHIO:
His name is Biondeling of your flesh?
What is the Duke of Norfolk stand not forgot.

ROMEO:
Nay, come away these birds day.

HORTENSIO:
'Clarde here, sir! my wife, the heavens, take thee these virtues
That you must lose your leisure.

Will Herenorant:
And the gods prevented the indict me attend.
Our argion, some subject sort now:
Now purpose this house, be you put down,
But I disdain: see where the world goes hither;
No, not for your content. You, timour toward the God.

TRANIO:
Sweet may of Norfolk, smoothed up bow. We'll put you to stand:
When you shall find my lands rot on the earth
When it is require by me. Here, Master Froth, father.
The will of wife, Paulina Lord Rivers are at night,
In pri